In [42]:
from collections import defaultdict
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (RandomForestClassifier,GradientBoostingClassifier)
from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.svm import LinearSVC

# Preparacion de datos

In [43]:
def get_postulaciones(size=None):
    postulaciones = pd.read_csv('data/FiubaHasta15Abril/fiuba_4_postulaciones.csv', nrows =size).drop_duplicates(subset=['idpostulante', 'idaviso'], keep='last')
    columns_rename = {'idaviso': 'id_aviso', 'idpostulante': 'id_postulante', 'fechapostulacion': 'fecha_postulacion'}
    postulaciones = postulaciones.rename(columns=columns_rename)
    postulaciones['fecha_postulacion']=pd.to_datetime(postulaciones['fecha_postulacion'])
    return postulaciones

def get_vistas(size=None):
    vistas1 = pd.read_csv('data/FiubaHasta15Abril/fiuba_3_vistas.csv', nrows =size)
    vistas2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_3_vistas.csv', nrows =size)
    vistas3 = pd.read_csv('data/fiuba_3_vistas.csv', nrows =size)
    vistas = pd.concat([vistas1, vistas2, vistas3])
    vistas_sumarizadas = vistas.groupby(['idpostulante', 'idAviso'], as_index=False)['timestamp'].count()
    columns_rename = {'idAviso': 'id_aviso', 'idpostulante': 'id_postulante', 'timestamp': 'visualizaciones'}
    vistas_sumarizadas = vistas_sumarizadas.rename(columns=columns_rename)
    
    return vistas_sumarizadas


def tokens(doc):
    return (tok.lower() for tok in re.findall(r"\w+", doc))


def token_freqs(doc):
    freq = defaultdict(int)
    for tok in tokens(doc):
        freq[tok] += 1
    return freq


def get_avisos_detalle():
    avisos1 = pd.read_csv('data/fiuba_6_avisos_detalle.csv')
    avisos2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_6_avisos_detalle.csv')
    avisos3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_6_avisos_detalle.csv')
    avisos4 = pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv')        
    avisos_detalle = pd.concat([avisos1,avisos2,avisos3,avisos4]).drop_duplicates(subset=['idaviso'], keep='last').reset_index(drop=True)
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    avisos_detalle = avisos_detalle.rename(columns=columns_rename)
    to_nivel_laboral_nro = {'Senior / Semi-Senior' : 2, 'Junior':1, 'Otro':0,
       'Jefe / Supervisor / Responsable':3,
       'Gerencia / Alta Gerencia / Dirección':4}
    to_tipo_trabajo_nro={'Full-time':0, 'Part-time':1, 'Teletrabajo':2, 'Por Horas':3, 'Pasantia':4,
       'Temporario':5, 'Por Contrato':6, 'Fines de Semana':7, 'Primer empleo':8,
       'Voluntario':9}
    to_nombre_area_numero = pd.Series(avisos_detalle['nombre_area'].unique()).to_dict()
    to_nombre_area_numero  = {v: k for k, v in to_nombre_area_numero.items()}
    avisos_detalle['nivel_laboral_nro']= avisos_detalle['nivel_laboral'].map(to_nivel_laboral_nro)
    avisos_detalle['tipo_de_trabajo_nro'] = avisos_detalle['tipo_de_trabajo'].map(to_tipo_trabajo_nro)
    avisos_detalle['nombre_area_nro'] = avisos_detalle['nombre_area'].map(to_nombre_area_numero)

    h = FeatureHasher(n_features = 40, input_type='string', dtype='float32')

    avisos_detalle['titulo_as_token_freq'] = avisos_detalle.titulo.apply(lambda x: token_freqs(x))
    #x = h.transform(avisos_detalle['titulo'])
    # en lugar de usar el titulo uso un token dict
    x = h.transform(avisos_detalle['titulo_as_token_freq'])
    avisos_detalle['titulo'] = list(x.toarray())

    titulos_como_lista = avisos_detalle.titulo.apply(pd.Series)
    avisos_detalle = pd.merge(avisos_detalle, titulos_como_lista, left_index = True, right_index = True)
    avisos_detalle = avisos_detalle.drop(['titulo'], axis=1)  

    return avisos_detalle

def get_year_of_birth(postulantes_genero_edad):
    return (pd.to_datetime
            (postulantes_genero_edad['fechanacimiento'], errors='coerce', format='%Y-%m-%d')
            .dt.year)

def get_age(yearOfBirth):
    return 2018 - yearOfBirth
    
def get_age_range(yearOfBirth):
    age = get_age(yearOfBirth)
    if(age<25): return 'Entre 18 y 24'
    if(age<30): return 'Entre 25 y 30'
    if(age<35): return 'Entre 30 y 35'
    if(age<40): return 'Entre 35 y 40'
    if(age<45): return 'Entre 40 y 45'
    if(age<50): return 'Entre 45 y 50'
    return 'Mayor de 50'

def get_order_for_age_range():
    return ['Entre 18 y 24', 'Entre 25 y 30','Entre 30 y 35','Entre 35 y 40','Entre 40 y 45','Entre 45 y 50', 'Mayor de 50']

def get_postulantes_nivel_educativo_para(path):
    postulantes_nivel_educativo = pd.read_csv(path)
    columns_rename = {'idpostulante': 'id_postulante', 'nombre': 'formacion_postulante', 'estado': 'estado_formacion_postulante'}
    postulantes_nivel_educativo=postulantes_nivel_educativo.rename(columns=columns_rename)
    formacion_to_number={'Secundario' : 10, 'Otro': 20, 'Terciario/Técnico' : 30, 'Universitario' : 40, 'Posgrado' : 50,
    'Master' : 50, 'Doctorado' : 50}
    postulantes_nivel_educativo['formacion_postulante_numero']=postulantes_nivel_educativo['formacion_postulante'].map(formacion_to_number);
    estado_to_number = {'En Curso': 4, 'Abandonado': 0, 'Graduado': 8}
    postulantes_nivel_educativo['estado_formacion_postulante_numero']=postulantes_nivel_educativo['estado_formacion_postulante'].map(estado_to_number)
    postulantes_nivel_educativo['nivel_educativo_postulante_numero'] = postulantes_nivel_educativo['formacion_postulante_numero'] + postulantes_nivel_educativo['estado_formacion_postulante_numero']
    postulantes_nivel_educativo['nivel_educativo_postulante_texto'] = postulantes_nivel_educativo['formacion_postulante'] + ' - ' + postulantes_nivel_educativo['estado_formacion_postulante']
    relevant_columns = ['id_postulante','nivel_educativo_postulante_texto', 'nivel_educativo_postulante_numero']
    postulantes_nivel_educativo = postulantes_nivel_educativo[relevant_columns]
    grouped=postulantes_nivel_educativo.groupby(['id_postulante']).agg({'nivel_educativo_postulante_numero':['max']}) 
    df=grouped.reset_index()
    df.columns = ['id_postulante', 'maximo_nivel_educativo_postulante']
    return df

def get_postulantes_nivel_educativo():
    postulantes1 = get_postulantes_nivel_educativo_para('data/fiuba_1_postulantes_educacion.csv')
    postulantes2 = get_postulantes_nivel_educativo_para('data/FiubaDesde15Abril/fiuba_1_postulantes_educacion.csv')
    postulantes3 = get_postulantes_nivel_educativo_para('data/FiubaHasta15Abril/fiuba_1_postulantes_educacion.csv')
    return pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['id_postulante'], keep='last').reset_index(drop=True)

def get_postulantes_genero_edad():
    postulantes1 = pd.read_csv('data/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes2 = pd.read_csv('data/FiubaDesde15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes3 = pd.read_csv('data/FiubaHasta15Abril/fiuba_2_postulantes_genero_y_edad.csv')
    postulantes_genero_edad = pd.concat([postulantes1,postulantes2,postulantes3]).drop_duplicates(subset=['idpostulante'], keep='last').reset_index(drop=True)
    postulantes_genero_edad['año_nacimiento_postulante']=get_year_of_birth(postulantes_genero_edad)
    postulantes_genero_edad['edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age, na_action=None)
    postulantes_genero_edad['rango_edad_postulante']=postulantes_genero_edad['año_nacimiento_postulante'].map(get_age_range, na_action=None)
    columns_rename = {'idpostulante': 'id_postulante', 'fechanacimiento': 'fecha_nacimiento_postulante', 'sexo': 'genero_postulante'}
    postulantes_genero_edad = postulantes_genero_edad.rename(columns=columns_rename)
    postulantes_genero_edad = postulantes_genero_edad[['id_postulante', 'genero_postulante', 'fecha_nacimiento_postulante', 'edad_postulante', 'rango_edad_postulante']]
    postulantes_genero_edad['genero_postulante_nro'] = postulantes_genero_edad['genero_postulante'].map({'FEM': 0, 'MASC': 1, 'NO_DECLARA': 2})
    return postulantes_genero_edad

def get_postulantes_limpios():
    postulantes = pd.merge(get_postulantes_genero_edad(), get_postulantes_nivel_educativo(), on='id_postulante', how='outer')
    order_for_columns = ['id_postulante','edad_postulante', 'genero_postulante', 'genero_postulante_nro', 'maximo_nivel_educativo_postulante']
    return postulantes[order_for_columns]

def get_detalle_postulaciones(size=None):
    postulaciones = get_postulaciones(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_postulaciones = pd.merge(postulantes, postulaciones, on='id_postulante', how='inner') 
    detalle_postulaciones = pd.merge(detalle_postulaciones, avisos, on='id_aviso', how='inner')
    
    return detalle_postulaciones

def get_detalle_vistas(size=None):
    vistas = get_vistas(size)
    avisos = get_avisos_detalle()
    postulantes = get_postulantes_limpios()
    detalle_vistas = pd.merge(postulantes, vistas, on='id_postulante', how='inner') 
    detalle_vistas = pd.merge(detalle_vistas, avisos, on='id_aviso', how='inner')
    return detalle_vistas

def x_entrenamiento():
    return ['edad_postulante', 'genero_postulante_nro', 'maximo_nivel_educativo_postulante', 'nivel_laboral_nro', 'visualizaciones', 'nombre_area_nro',0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39]

def x_categoricas():
    return ['genero_postulante', 'maximo_nivel_educativo_postulante', 'nivel_laboral', 'tipo_de_trabajo', 'nombre_area']

def x_humanas():
    return ['id_postulante', 'id_aviso', 'titulo', 'descripcion', 'denominacion_empresa'] + x_categoricas

def y_entrenamiento():
    return ['sepostulo']

def columnas_relevantes_test_data():
    return ['id','id_aviso','id_postulante'] + x_entrenamiento()

def get_test_data():
    tests = pd.read_csv('data/Test/test_final_100k.csv')
    columns_rename = {'idpostulante': 'id_postulante', 'idaviso': 'id_aviso'}
    tests = tests.rename(columns=columns_rename)
    tests = pd.merge(tests, get_avisos_detalle(), on='id_aviso', how='left')
    tests = pd.merge(tests, get_postulantes_limpios(), on='id_postulante', how='left')    
    vistas = get_vistas()
    tests = pd.merge(tests, vistas, on=['id_postulante', 'id_aviso'], how='left' )    
    tests['visualizaciones'].fillna(value=0, inplace=True) 
    tests['edad_postulante'].fillna(value=0, inplace=True)
    tests['maximo_nivel_educativo_postulante'].fillna(value=0, inplace=True)
    tests['nivel_laboral_nro'].fillna(value=0, inplace=True)
    tests['nombre_area_nro'].fillna(value=0, inplace=True)
                                                  
    return tests[columnas_relevantes_test_data()]

def get_default_null_values(df):
    return {'edad_postulante':int(df['edad_postulante'].mean()), 'genero_postulante_nro':int(0), 'maximo_nivel_educativo_postulante':int(df['maximo_nivel_educativo_postulante'].mean()), 'nivel_laboral_nro':int(0), 'visualizaciones':int(0), 'nombre_area_nro':int(0)}


def get_test_data_clean():
    tests = get_test_data()
    tests = tests.fillna(value=get_default_null_values(tests))
    return tests

def get_datos_entrenamiento(size=None):
    postulaciones_aplicadas = get_detalle_postulaciones(size)
    columnas_relevantes = x_entrenamiento() + y_entrenamiento()
    postulaciones_aplicadas['sepostulo'] = True
    postulaciones_no_aplicadas = get_detalle_vistas(size)
    vistas = postulaciones_no_aplicadas[['id_postulante', 'id_aviso', 'visualizaciones']]        
    postulaciones_aplicadas = pd.merge(postulaciones_aplicadas, vistas, on=['id_postulante', 'id_aviso'], how='left' )    
    postulaciones_aplicadas['visualizaciones'].fillna(value=0, inplace=True)    
    postulaciones_no_aplicadas['sepostulo'] = False    
    postulaciones_aplicadas = postulaciones_aplicadas[:postulaciones_no_aplicadas.shape[0]]
    postulaciones_no_aplicadas = postulaciones_no_aplicadas[:postulaciones_aplicadas.shape[0]]
    candidatos = postulaciones_aplicadas.append(postulaciones_no_aplicadas).drop_duplicates(subset=['id_aviso', 'id_postulante'], keep='first')[columnas_relevantes].dropna()
    candidatos_true  = candidatos[candidatos['sepostulo']==True]
    candidatos_false = candidatos[candidatos['sepostulo']==False]
    candidatos_true  = candidatos_true[:candidatos_false.shape[0]]
    candidatos_false = candidatos_false[:candidatos_true.shape[0]]
    return candidatos_false.append(candidatos_true)


def get_predictor(set_entrenamiento):
    # Prepare feature and dependent variable along with categorical encoding
    X=pd.get_dummies(set_entrenamiento.loc[:, x_entrenamiento()])
    y=set_entrenamiento.loc[:, 'sepostulo']
    clf = svm.SVC()
    return clf.fit(X, y)

def get_random_forest_predictor(set_entrenamiento):
    X = set_entrenamiento.loc[:, x_entrenamiento()]
    Y = set_entrenamiento.loc[:, 'sepostulo']
    clf = RandomForestClassifier(n_estimators=15)
    return clf.fit(X, Y)

def get_sgd_classifier(set_entrenamiento):
    X = set_entrenamiento.loc[:, x_entrenamiento()]
    Y = set_entrenamiento.loc[:, 'sepostulo']
    clf = linear_model.SGDClassifier()
    return clf.fit(X, Y)

def get_general_predictor():
    return get_random_forest_predictor(get_datos_entrenamiento(10000))

def get_postulantes_a_personalizar():
    tests = pd.read_csv('data/Test/test_final_100k.csv')
    return set(tests['idpostulante'])

def get_personalized_predictor(cantidad_mininima_observaciones):
    datos_entrenamiento = get_datos_entrenamiento()
    postulantes = get_postulantes_a_personalizar()
    return get_random_forest_predictor(get_datos_entrenamiento(10000))

#El ensamble
def get_predictor_machine_avisos(cantidad_mininima_observaciones):
    machine = { generalPredictor : get_general_predictor(), personalizedPredictor : get_personalized_predictor(cantidad_mininima_observaciones)} 
    return machine

def obtener_predicciones(predictor, set_test):
    return predictor.predict(set_test[x_entrenamiento()])

def get_score(dataset, clasificador):
    X_train, X_test, y_train, y_test = train_test_split(dataset.loc[:, x_entrenamiento()], dataset.loc[:, 'sepostulo'], test_size=0.33, random_state=42)
    clasificador = clasificador.fit(X_train, y_train)    
    return clasificador.score(X_test, y_test)
 

In [50]:
set_entrenamiento = get_datos_entrenamiento(10000)

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


In [45]:
set_entrenamiento['sepostulo'].value_counts()

True     869
False    869
Name: sepostulo, dtype: int64

In [10]:
set_entrenamiento.shape

(6098918, 47)

In [11]:
set_entrenamiento.head()

,edad_postulante,genero_postulante_nro,maximo_nivel_educativo_postulante,nivel_laboral_nro,visualizaciones,nombre_area_nro,0,1,3,4,...,31,32,33,34,35,36,37,38,39,sepostulo
1,39.0,1.0,48.0,2.0,1.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
2,36.0,0.0,44.0,2.0,1.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
3,34.0,0.0,40.0,2.0,1.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
4,33.0,0.0,48.0,2.0,2.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False
5,28.0,1.0,44.0,2.0,4.0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,False


# Armado de los sets de test y de entrenamiento

In [51]:
from sklearn.model_selection import train_test_split

x = set_entrenamiento.columns.values.tolist()
x.remove('sepostulo')

X_train, X_test, y_train, y_test = train_test_split(set_entrenamiento.loc[:, x], set_entrenamiento.loc[:, 'sepostulo'], test_size=0.3, random_state=43)


# Armado de una grilla de entrenamiento

In [ ]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [5, 10, 15, 20], 'max_depth': [2, 5, 7, 9],
                     'min_samples_split': [5,10,15], 'min_samples_leaf': [5,10,15]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision



# Entrenamiento

In [36]:
clf = RandomForestClassifier(n_estimators=50,min_samples_split=10,min_samples_leaf=10)

predictor =  clf.fit(X_train, y_train)


In [37]:
predictor.score(X_test, y_test)

0.8491317588469215

#### Prueba y resultados para la competencia

In [35]:
def obtener_predicciones(predictor, set_test):
    x = set_entrenamiento.columns.values.tolist()
    x.remove('sepostulo')    
    return predictor.predict(set_test[x]).astype(int)

def guardar_resultados(fileName, predictor, set_test):
    result = obtener_predicciones(predictor, set_test)
    set_test['sepostulo'] = result
    set_test[['id','sepostulo']].set_index('id').to_csv(fileName)
    return

set_test = get_test_data()

guardar_resultados('100K ultimo.csv', predictor, set_test)
set_test['sepostulo'].value_counts()

/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/hashing.py:94: DeprecationWarning: the option non_negative=True has been deprecated in 0.19 and will be removed in version 0.21.
  " in version 0.21.", DeprecationWarning)


1    71585
0    28415
Name: sepostulo, dtype: int64